In [1]:
import os
import json
import re

from langchain_openai import AzureOpenAIEmbeddings
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents._generated.models import QueryCaptionResult

from openai import AzureOpenAI
import tiktoken

from dotenv import load_dotenv

load_dotenv()

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

class AzureEmbeddings:

    def __init__(self):
        pass

    @staticmethod
    def get_embedding():
        return AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

    @staticmethod
    def generate_embeddings(content: str):
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

        doc_result = embeddings.embed_documents([content])

        return doc_result[0]

openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"), 
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    )

embeddings_client = AzureEmbeddings()
store_search_url: str = f'https://{os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME')}.search.windows.net'
search_client = SearchClient(
            store_search_url, os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME"),
            AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"))
        )

In [2]:
# user_input = "¿Qué dice el artículo 103 del Código Penal?"  # 1
# user_input = "¿Cuáles son las causales de agravación del homicidio?"  # 2
# user_input = "Si A le roba el celular a B, ¿qué delito comete?"  # 3
# user_input = "¿Cuál es la diferencia entre dolo eventual y culpa con representación?"  # 4
# user_input = "¿Cuáles son las formas de autoría en derecho penal?"  # 5
# user_input = "¿Cuándo se consuma el hurto?"  # 6
# user_input = "¿Las personas jurídicas pueden ser sujetos pasivos de los delitos de injuria y calumnia?"  # 7
# user_input = "¿Cuál es el término que tiene la Fiscalía para formular acusación?"  # 8
# user_input = "¿Se puede archivar una investigación cuando se encuentra que el procesado actuó en legítima defensa?"  # 9
# user_input = "¿Se puede precluir una investigación antes de formular imputación?"  # 10
# user_input = "¿Cuáles son las causales de ausencia de responsabilidad en el derecho penal?"  # 11
# user_input = "¿Cuáles son las causales genéricas de atenuación punitiva?"  # 12
# user_input = "¿Qué se entiende por “documento” en el derecho penal?"  # 13
# user_input = "¿Cuáles son las causales de agravación de la estafa?"  # 14
# user_input = "¿Se pueden realizar preguntas sugestivas en el contrainterrogatorio del juicio oral?" #15
# user_input = "¿Un juez debe ordenar la captura del procesado cuando anuncie sentido de fallo condenatorio?" #16
# user_input = "¿Cuáles son las fuentes de posición de garante según el código penal?" #17
# user_input = "¿La búsqueda selectiva en base de datos requiere control previo?" #18
# user_input = "¿Cuándo procede la captura en flagrancia?" #19
# user_input = "Si alguien falsifica un documento y lo usa como medio para engañar a otra persona y llevarlo a que le pague algo que no le debe, ¿qué delito(s) comete?" #20

# user_input = "Cuál es la sanción por no presentar la declaración de renta a timepo?"
# user_input = "Qué es el derecho a la vida?"
# user_input = "En qué artículo de la constitución está consagrado el derecho a la vida?"
# user_input = "En qué artículo del código penal se contempla la definición de documento?"
# user_input = "¿Qué dice el artículo 14 del código penal?"
# user_input = "¿Qué dice el artículo 14 de la ley 599 del 2000?"
# user_input = "¿Dónde se considera realizada la conducta punible?"
# user_input = "¿Qué dice el artículo 241 del código penal?"
# user_input = "¿Qué se entiende por “documento” en el código penal?"
# user_input = "¿Qué dice el artículo 249 del código penal?"
# user_input = "¿Qué dice el artículo 221 del código penal?"
# user_input = "¿En qué artículo se encuentra el delito de calumnia?"
# user_input = "Si B imputa falsamente a A un delito que no ha cometido ¿En qué conducta punible incurre?"
# user_input = "En qué delito incurre una persona que entre sin autorización en un sistema informático?"
# user_input = "Daniel Andrés Fúquenes Barriga, en su condición de auxiliar de la justicia y secuestre, recaudó una suma de dinero por concepto de arrendamiento, específicamente $684,000 entre febrero y julio de 2015. Este dinero debía ser entregado a su dueño o poseedor, pero Fúquenes Barriga retuvo la suma para sí mismo, incumpliendo con la obligación de devolverla. Este acto de retención y apropiación del dinero, que se le había confiado por un título no traslativo de dominio, ¿Qué delito cometió?"
# user_input = "¿Qué delito comtió una persona que adquirió un bien cuyo origen inmediato es una actividad ilícita?"
# user_input = "¿Qué dice el artículo 323 del código penal?"
# user_input = "¿Qué artículo contiene el delito de lavado de activos en el código penal?"
# user_input = "¿Qué delito comete un juez que profiere una sentencia manifiestamente contraria a la ley?"
# user_input = "Si A dispara a B en Barrranquilla pero B se muere en Cartagena, ¿Dónde se entiende realizado el homicidio?"
# user_input = "Según el artículo 14 del código penal si A dispara a B en Barrranquilla pero B se muere en Cartagena, ¿Dónde se entiende realizado el homicidio?"
user_input = "¿Qué se entiende por servido público en código penal?"

print("🧑 User input:")
print(user_input)

🧑 User input:
¿Qué se entiende por servido público en código penal?


In [3]:
conversation = []
query_system_template = "You are a legal assistant and your goal is to generate a search query for legal documents. You have access to Azure AI Search index with 100's of documents. Generate a search query in Spanish based on what the user asked you. Your query should always contain at least one and no more than two laws, articules and/or documents titles. The query will be used in a hybrid search (semantic and full text) so keep it short but make sure to include all the main keywords and actions needed to give the necesary context to the query. In the query you should include at least one concept followed by one specific source. eg: 'homicidio artículo 103 ley 599 del 2000'. Don't include more than 1 source. Sometimes the user will tell you the exact source you should query, but if you can't find the source out of what the user asked you, you can use the tool call 'get_supporting_info' to identify sources that could help you generate a better query with the user question. You response will be in a JSON format following this format: {'query_type':'<query_type>', 'query':'<query>'} For the 'query_type' you will choose between 'specific' if the user asks for a specific law, article or title, or 'interpretation' if the user didn't asked for something specific but rather need help making an interpretation of the law. .Don't include the tags ```json at the begenning nor the end"
system_message = [{"role": "system", "content": query_system_template}]


query_user_content_template = f"Generate search query for: {user_input}."
user_message = [{"role": "user", "content": query_user_content_template}]

# query_messages = [
#     {"role": "system", "content": query_system_template},
#     ,
# ]

In [4]:
get_supporting_info = {
    "type": "function",
    "function": {
        "name": "get_supporting_info",
        "description": "Use this tool to get laws, articles and titles to generate a better query. If the user already says what the source or where to look at, don't use this tool.",
        "parameters": {},
    },
}
get_next_chunk = {
    "type": "function",
    "function": {
        "name": "get_next_chunk",
        "description": "Use this tool to get when an important sources comes incomplete or the content is cut off. This tool will help you retrieve the following section of that given source.",
        "parameters": {
            "type": "object",
            "properties": {
                "source_id": {
                    "type": "string",
                    "description": "the unique identifier of the chunk that is incomplete. eg:'20240719192458csjscpboletinjurisprudencial20181219pdf_chunk30'",
                },
            },
            "required": ["source_id"],
        },
    },
}

In [5]:
import re

query_completion = openai_client.chat.completions.create(
    tools=[get_supporting_info],
    messages=system_message + user_message,
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0.0,
    n=1,
)

supporting_content = []
printer_supporting_content = []
query_vectors = []

if query_completion.choices[0].finish_reason == "tool_calls":
    tool_call = query_completion.choices[0].message.tool_calls[0]
    query_vectors = embeddings_client.generate_embeddings(content=user_input)
    results_supporting_content = search_client.search(
        search_text=user_input,
        # filter=filters,
        top=3,
        query_type=QueryType.SEMANTIC,
        vector_queries=[
            {
                "vector": query_vectors,
                "k": 3,
                "fields": "content_vector",
                "kind": "vector",
                "exhaustive": True,
            }
        ],
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
        scoring_profile="legal",
        scoring_parameters=[
            "tagx6-Constitucional",
            "tagx5-Legal",
            "tagx4-Infralegal",
            "tagx3-Jurisprudencia",
            "tagx2-Doctrina",
        ],
    )

    results_supporting_content_top = list(results_supporting_content)[:3]

    for document in results_supporting_content_top:
        doc = {
            # "title": document["title"],
            # "keywords": document["keywords"],
            # "category": document["category"],
            "content": document["content"],
        }
        supporting_content.append(doc)
        printer_supporting_content.append(json.dumps(doc, indent=4))

    new_query_user_content_template = f"Generate search query for: {user_input}. Use these supporting documents to identify any useful source likw articles, laws or document titles to include in the query: {supporting_content}"

    tool_messages = [
        {
            "role": "assistant",
            "tool_calls": [
                {
                    "id": tool_call.id,  # call_rb1Don1fEb6oLpN7rHmBh4RW
                    "function": {
                        "arguments": "{}",
                        "name": "get_supporting_info",
                    },
                    "type": "function",
                }
            ],
        },
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": "get_supporting_info",
            "content": str(supporting_content),
        },
    ]
    new_query_messages = system_message + user_message + tool_messages
    conversation = user_message + tool_messages

    # print(new_query_messages)
    new_query_completion = openai_client.chat.completions.create(
        messages=new_query_messages,
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        temperature=0.0,
        n=1,
    )
    query = new_query_completion.choices[0].message.content
    query_string = query.replace("'", '"')
    query_data = json.loads(query_string)
    query_vectors = embeddings_client.generate_embeddings(content=query_data["query"])
    print("Type: ", query_data["query_type"])
    print("Query: ", query_data["query"])
    print("=========")
    print("\n".join(printer_supporting_content))
else:
    print("Else")
    query = query_completion.choices[0].message.content
    print("Query competion: ", query_completion)
    query_string = query.replace("'", '"')
    query_data = json.loads(query_string)
    query_vectors = embeddings_client.generate_embeddings(content=query_data["query"])
    print("Type: ", query_data["query_type"])
    print("Query: ", query_data["query"])

Type:  specific
Query:  servidor público artículo 20 Ley 599 de 2000
{
    "content": "Cristhian Zambrano G\u00f3mez 87\nII. VIOLENCIA CONTRA SERVIDOR P\u00daBLICO\nEl delito de violencia contra servidor p\u00fablico, consagrado en el art\u00edculo\n429 del C\u00f3digo Penal, advierte que la persona que \u201cejerza violencia contra\nservidor p\u00fablico por raz\u00f3n de sus funciones o para obligarlo a ejecutar\nu omitir alg\u00fan acto propio de su cargo o a realizar uno contrario a sus\ndeberes oficiales, incurrir\u00e1 en prisi\u00f3n de cuatro (4) a ocho (8) a\u00f1os de\nprisi\u00f3n\u201d4. En este tipo penal, el sujeto activo \u2013quien realiza la conducta\u2013\nes indeterminado, es decir, no debe tener una calidad espec\u00edfica y la acci\u00f3n\npuede ser ejecutada por cualquier persona. No obstante, el tipo penal exige\nque el sujeto pasivo \u2013sobre quien recae la conducta\u2013 sea cualificado, es\ndecir, debe ser un servidor p\u00fablico, requisito esencial para su

In [6]:
def search_specific():
    print("Specific search for:")
    print(query_data["query"])
    return search_client.search(
        search_text=query_data["query"],
        # filter=filters,
        top=50,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
        scoring_profile="legal",
        scoring_parameters=[
            "tagx6-Constitucional",
            "tagx5-Legal",
            "tagx4-Infralegal",
            "tagx3-Jurisprudencia",
            "tagx2-Doctrina",
        ],
    )


def search_interpretation():
    print("Interpretation search for:")
    print(query_data["query"])
    return search_client.search(
        search_text=query_data["query"],
        # filter=filters,
        top=50,
        query_type=QueryType.SEMANTIC,
        vector_queries=[
            {
                # "vector": user_input_vectors,
                "vector": query_vectors,
                "k": 50,
                "fields": "content_vector",
                "kind": "vector",
                "exhaustive": True,
            }
        ],
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
        scoring_profile="legal",
        scoring_parameters=[
            "tagx6-Constitucional",
            "tagx5-Legal",
            "tagx4-Infralegal",
            "tagx3-Jurisprudencia",
            "tagx2-Doctrina",
        ],
    )


# Fields:
# 1. Título primero: 10
# 4. Año: 9
# 3. Autor: 8
# 5. Categoría: 7
# 6. Keywords: 6
# 2. Contenido: 3

# Category:
# "tagx6-Constitucional",
# "tagx5-Legal",
# "tagx4-Infralegal",
# "tagx3-Jurisprudencia",
# "tagx2-Doctrina",

results = []
if query_data["query_type"] == "specific":
    results = search_specific()
elif query_data["query_type"] == "interpretation":
    results = search_interpretation()


def add_to_context(results, token_limit=3500):
    printer_documents = []

    results_list = list(results)

    top_results = results_list[:5]
    supporting_results = results_list[5:]

    documents = []

    for document in top_results:
        # captions: QueryCaptionResult = document["@search.captions"]
        # print(document)
        doc_formatted = {
            # "score": document["@search.score"],
            # "rerank": document["@search.reranker_score"],
            "id": document["id"],
            "content": document["content"],
            "external_id": document["external_id"],
            "title": document["title"],
            "author": document["author"],
            "keywords": document["keywords"],
            "category": document["category"],
            "year": document["year"],
        }
        documents.append(doc_formatted)
        printer_documents.append(json.dumps(doc_formatted, indent=4))

    current_token_count = num_tokens_from_string(str(documents))

    additional_documents = []
    for document in supporting_results:
        captions: QueryCaptionResult = document["@search.captions"]
        captions_text = " // ".join([caption.text for caption in captions])
        # print(document)
        doc_formatted = {
            # "score": document["@search.score"],
            # "rerank": document["@search.reranker_score"],
            "id": document["id"],
            "captions": captions_text,
            "external_id": document["external_id"],
            "title": document["title"],
            "author": document["author"],
            "keywords": document["keywords"],
            "category": document["category"],
            "year": document["year"],
        }

        # Tema de contexto para responder la respuesta en base a qué
        # pensar en enrutar los queries dependiendo del tema, del keyword.

        new_token_count = current_token_count + num_tokens_from_string(
            str(doc_formatted)
        )

        if new_token_count <= token_limit:
            additional_documents.append(doc_formatted)
            printer_documents.append(json.dumps(doc_formatted, indent=4))
            current_token_count = new_token_count
        else:
            break

    response = {
        "sources": documents + additional_documents,
        "documents": documents,
        "printer_documents": printer_documents,
        "additional_documents": additional_documents,
        "token_count": current_token_count,
    }

    return response


sources = add_to_context(results)
documents = sources["sources"]

print("Legal profile - Query for: ")
print(query)
print("Num of docs:", len(documents), " ---  Tokens: ", sources["token_count"])
print("=============================")
print("\n".join(sources["printer_documents"]))

Specific search for:
servidor público artículo 20 Ley 599 de 2000
Legal profile - Query for: 
{
  'query_type': 'specific',
  'query': 'servidor público artículo 20 Ley 599 de 2000'
}
Num of docs: 9  ---  Tokens:  3334
{
    "id": "20240719201656ley599de2000pdf_chunk308",
    "content": "LEY 599 DE 2000 C\u00d3DIGO PENAL DE COLOMBIA\nsubsidio o ayuda de una entidad p\u00fablica a la finalidad\na la cual est\u00e9n destinados.\nCAPITULO SEGUNDO\nDE LA CONCUSI\u00d3N\nArt\u00edculo 404.Concusi\u00f3n. El servidor p\u00fablico que\nabusando de su cargo o de sus funciones constri\u00f1a o\ninduzca a alguien a dar o prometer al mismo servidor\no a un tercero, dinero o cualquier otra utilidad\nindebidos, o los solicite, incurrir\u00e1 en prisi\u00f3n de\nnoventa y seis (96) a ciento ochenta (180) meses,\nmulta de sesenta y seis punto sesenta y seis (66.66)\na ciento cincuenta (150) salarios m\u00ednimos legales\nmensuales vigentes, e inhabilitaci\u00f3n para el\nejercicio de derechos y funci

In [7]:
response_system_template = f'Eres Ariel, un asistente para la investigación legal. Sé lo más detallado y preciso posible en tus respuestas. Cita el máximo número de fuentes posible, sin salirte del tema. En caso de encontrar información contradictoria, señálala y sugiere una posible causa. Expresa toda la información que encuentres en las fuentes proporcionadas por el usuario. Solamente si la respuesta no está en las fuentes proporcionadas, responde "No encuentro información con esos términos, ¿puedes reformular tu consulta?". Las fuentes proporcionadas por el usuario son fragmentos de máximo 500 tokens del documento completo, así que en ocasionas las fuentes proporcionadas van a tener contenido incompleto o cortado, si ves que la fuente más importante no está completa o puede que el documento completo tenga más información relevante, ejecuta "get_next_chunk" para obtener el siguiente fragmento de esa fuente. Interpreta las fuentes dependiendo de su categoría: 1. Constitucional: Es la norma máxima y más importante establecida por el gobierno del país. 2. Legal: Es la que le sigue en importancia y son los decretos, códigos y leyes oficiales del país establecida por los ministerios. 3. Infralegal: Le sigue en importancia a la categoría Legal. 4. Jurisprudencia: Son los documentos de juicios reales y de cómo se ha ejercido la ley en el pasado. 5. Doctrina: Son documentos o libros escritos por autores del derecho que no son parte de la ley pero ayudan a darle una interpetación a la ley. Es Al incluir títulos en tu respuesta, usa formato html (ej: titulos, <strong>). Recibirás las fuente n un formato json donde encontrarás el contenido de la fuente, el título del documento, entre otra información de utilidad, siempre incluye el título de la fuente para cada hecho que uses en la respuesta. Todas las fuentes son PDFs. Utiliza corchetes para referenciar la fuente, por ejemplo [info1.pdf]. No combines fuentes, lista cada fuente por separado, por ejemplo [info1.txt] [info2.pdf]. El formato de algunas fuentes puede incluir acentos, puntos o guiones. No manipules los id de las fuentes ya que ellos son importantes para el funcionamiento de esta conversación. Escribe la respuesta en formato HTML, pero sin incluir los tags "```html" al principio o final principio de tu respuesta.'
# response_system_template = f'Eres Ariel, un asistente para la investigación legal. Sé lo más detallado y preciso posible en tus respuestas. Cita el máximo número de fuentes posible, sin salirte del tema. En caso de encontrar información contradictoria, señálala y sugiere una posible causa. Expresa toda la información que encuentres en las fuentes proporcionadas por el usuario. Solamente si la respuesta no está en las fuentes proporcionadas, responde "No encuentro información con esos términos, ¿puedes reformular tu consulta?". En ocasiones las fuentes proporcionadas van contener contenido incompleto que sirven para responder la pregunta, solo en el caso que veas que una fuente proporcionada está incompleta y es la que necesitas para generar tu respuesta, ejecuta el tool "get_next_chunk" para obtener el siguiente fragmento de esa fuente, no ejecutes esta tool con las fuentes que tienen "captions". Interpreta las fuentes dependiendo de su categoría: 1. Constitucional: Es la norma máxima y más importante establecida por el gobierno del país. 2. Legal: Es la que le sigue en importancia y son los decretos, códigos y leyes oficiales del país establecida por los ministerios. 3. Infralegal: Le sigue en importancia a la categoría Legal. 4. Jurisprudencia: Son los documentos de juicios reales y de cómo se ha ejercido la ley en el pasado. 5. Doctrina: Son documentos o libros escritos por autores del derecho que no son parte de la ley pero ayudan a darle una interpetación a la ley. Es Al incluir títulos en tu respuesta, usa formato html (ej: titulos, <strong>). Recibirás las fuente n un formato json donde encontrarás el contenido de la fuente, el título del documento, entre otra información de utilidad, siempre incluye el título de la fuente para cada hecho que uses en la respuesta. Todas las fuentes son PDFs. Utiliza corchetes para referenciar la fuente, por ejemplo [info1.pdf]. No combines fuentes, lista cada fuente por separado, por ejemplo [info1.txt] [info2.pdf]. El formato de algunas fuentes puede incluir acentos, puntos o guiones. No manipules los id de las fuentes ya que ellos son importantes para el funcionamiento de esta conversación. Escribe la respuesta en formato HTML, pero sin incluir los tags "```html" al principio o final principio de tu respuesta.'

response_system_message = [{"role": "system", "content": response_system_template}]

response_few_shots = []

response_user_content_template = f"{user_input} Sources: {documents}"
response_user_message = [{"role": "user", "content": response_user_content_template}]

response_messages = response_system_message + response_user_message

# print(response_messages)

In [8]:
response = openai_client.chat.completions.create(
    tools=[get_next_chunk],
    messages=response_messages,
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0.2,
    n=1,
)

text_response = response.choices[0].message.content

In [9]:
def get_next_chunk_func(id):
    pattern = r"^(.*_chunk)(\d+)$"
    match = re.search(pattern, id)

    if match:
        prefix = match.group(1)
        chunk_number = int(match.group(2))
        return search_client.search(
            filter=f"id eq '{prefix}{chunk_number + 1}' or id eq '{prefix}{chunk_number + 2}'",
            top=2,
        )

In [10]:
print(user_input)
print("===============")
print(text_response)
print("")
print("===============")
print(response)
print("===============")
print(response_messages)

¿Qué se entiende por servido público en código penal?
None

ChatCompletion(id='chatcmpl-9oJ4zVQztDnZtG7d96qxpfOsJm96H', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_5JC9PoneNeGAc9QLT3S5nlJk', function=Function(arguments='{"source_id":"20240719201656ley599de2000pdf_chunk308"}', name='get_next_chunk'), type='function')]), content_filter_results={})], created=1721776769, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_abc28019ad', usage=CompletionUsage(completion_tokens=28, prompt_tokens=3628, total_tokens=3656), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])
[

In [11]:
# next_chunks = list(get_next_chunk("20240719201656ley599de2000pdf_chunk234"))
# print(next_chunks)

if response.choices[0].finish_reason == "tool_calls":
    tool_call = response.choices[0].message.tool_calls[0]
    tool_args_data = json.loads(tool_call.function.arguments)
    print("Args: ", tool_args_data)
    results_next_chunks = list(get_next_chunk_func(tool_args_data["source_id"]))
    print("Next chunks: ", results_next_chunks)

    next_chunks_docs = []
    printer_next_chunks_docs = []
    for document in results_next_chunks:
        doc_formatted = {
            "id": document["id"],
            "content": document["content"],
            "external_id": document["external_id"],
            "title": document["title"],
            "author": document["author"],
            "keywords": document["keywords"],
            "category": document["category"],
            "year": document["year"],
        }
        next_chunks_docs.append(doc_formatted)
        printer_next_chunks_docs.append(json.dumps(doc_formatted, indent=4))

    response_tool_messages = [
        {
            "role": "assistant",
            "tool_calls": [
                {
                    "id": tool_call.id,  # call_rb1Don1fEb6oLpN7rHmBh4RW
                    "function": {
                        "arguments": str(tool_args_data),
                        "name": "get_next_chunk",
                    },
                    "type": "function",
                }
            ],
        },
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": "get_next_chunk",
            "content": str(next_chunks_docs),
        },
    ]

    conversation = response_user_message + response_tool_messages

    new_response_messages = (
        response_system_message + response_user_message + response_tool_messages
    )
    response = openai_client.chat.completions.create(
        messages=new_response_messages,
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        temperature=0.2,
        n=1,
    )

    text_response = response.choices[0].message.content

    print("")
    print("Converastion: ")
    print(new_response_messages)
    print("")

Args:  {'source_id': '20240719201656ley599de2000pdf_chunk308'}
Next chunks:  [{'content': 'LEY 599 DE 2000 CÓDIGO PENAL DE COLOMBIA\nNota: Penas aumentadas por el artículo 33 Ley 1474 de\n2011: …Los tipos penales de que tratan los artículos\n406… de la Ley 599 de 2000 les será aumentada la pena\nde una sexta parte a la mitad cuando la conducta sea\ncometida por servidor público que ejerza como\nfuncionario de alguno de los organismos de control\ndel Estado. (vigencia a partir del 12 de julio de\n2011)\nArtículo 407.Cohecho por dar u ofrecer. El que dé u\nofrezca dinero u otra utilidad a servidor público, en\nlos casos previstos en los dos artículos anteriores,\nincurrirá en prisión de cuarenta y ocho (48) a ciento\nocho (108) meses, multa de sesenta y seis punto\nsesenta y seis (66.66) a ciento cincuenta (150)\nsalarios mínimos legales mensuales vigentes, e\ninhabilitación para el ejercicio de derechos y\nfunciones públicas de ochenta (80) a ciento cuarenta\ny cuatro (144) meses.\nNOTA

In [12]:
print(query_data)
print(user_input)
print("===============")
print("===============")
print("")
print(text_response)
print("")
print("===============")
print("===============")
print(response)


# ¿Dónde se considera realizada la conducta punible?
# Tenía que incluid lugar no momento:
# {'query_type': 'specific', 'query': 'conducta punible artículo 26 Código Penal'}

{'query_type': 'specific', 'query': 'servidor público artículo 20 Ley 599 de 2000'}
¿Qué se entiende por servido público en código penal?

<strong>Definición de Servidor Público en el Código Penal Colombiano</strong>

El Código Penal de Colombia, Ley 599 de 2000, menciona en varios artículos el término "servidor público" en el contexto de diferentes delitos y sanciones. Sin embargo, no proporciona una definición explícita y directa de lo que se entiende por "servidor público" en un solo artículo. A continuación, se presentan algunos ejemplos de cómo se utiliza el término en el Código Penal:

1. **Concusión**:
   <em>"El servidor público que abusando de su cargo o de sus funciones constriña o induzca a alguien a dar o prometer al mismo servidor o a un tercero, dinero o cualquier otra utilidad indebidos, o los solicite..."</em> [Ley 599 de 2000, Artículo 404] [20240719201656ley599de2000pdf_chunk308].

2. **Cohecho Propio**:
   <em>"El servidor público que reciba para sí o para otro, dine